<a href="https://colab.research.google.com/github/LinzStearns/CO_real_estate/blob/main/Colorado_RE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-26 20:54:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-02-26 20:54:11 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://gonbri-corealestate.s3.amazonaws.com/clean_housing_v2.csv"
spark.sparkContext.addFile(url)
clean_housing_df = spark.read.csv(SparkFiles.get("clean_housing_v2.csv"), sep=",", header=True, inferSchema=True)

In [5]:
#Show DataFrame
clean_housing_df.show()

+------------+----------+-----------+---------------------+--------------------+---------+----------+
|    SCHEDNUM|SITUS_CITY|SITUS_STATE|APPRAISED_TOTAL_VALUE|          D_CLASS_CN|SALE_YEAR|SALE_PRICE|
+------------+----------+-----------+---------------------+--------------------+---------+----------+
| 20218093000|    DENVER|         CO|               319900|RESIDENTIAL-ROWHOUSE|     2021|    405000|
|228310000000|    DENVER|         CO|               365000|RESIDENTIAL-CONDO...|     2020|    393500|
|609302000000|    DENVER|         CO|               505800|RESIDENTIAL-ROWHOUSE|     2019|    513000|
|609201000000|    DENVER|         CO|               400600|RESIDENTIAL-CONDO...|     2020|    395000|
|234544000000|    DENVER|         CO|               509300|RESIDENTIAL-CONDO...|     2018|    515000|
|705401000000|    DENVER|         CO|               252400|RESIDENTIAL-CONDO...|     2021|    284000|
|229329000000|    DENVER|         CO|               916700|RESIDENTIAL-ROWHOUSE|  

In [6]:
mode = "append"
jdbc_url="jdbc:postgresql://corealestatedb.c3ru0tkydktd.us-east-1.rds.amazonaws.com:5432/Colorado_RE"
config = {"user":"Colorado",
          "password": "realestate",
          "driver":"org.postgresql.Driver"}

In [7]:
# Write DataFrame to active_user table in RDS
clean_housing_df.write.jdbc(url=jdbc_url, table='clean_housing', mode=mode, properties=config)